In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from comm_agents.data.datahandler import RefExpDataset
from comm_agents.models.model_single_enc import SingleEncModel
import pandas as pd
import plotly.graph_objects as go


In [3]:
MODEL_PATH = './models/single_enc_model_2020-10-19.pt'

In [6]:
dataset = RefExpDataset()
model = SingleEncModel(observantion_size=dataset.observations.shape[1],
                       lat_space_size=3,
                       question_size=2,
                       enc_num_hidden_layers=20,
                       enc_hidden_size=100,
                       dec_num_hidden_layers=20,
                       dec_hidden_size=100,
                       num_decoding_agents=4)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [7]:
answers, lat_spaces, selection_biases = model(dataset.observations, dataset.questions)

In [8]:
df_answers = pd.DataFrame(answers.detach().numpy(),
                          columns=['alpha1', 'alpha2', 'phi1', 'phi2'])
df_answers.head()

,alpha1,alpha2,phi1,phi2
0,0.287824,0.286768,1.796131,1.795678
1,0.287824,0.286768,1.796131,1.795678
2,0.287824,0.286768,1.796131,1.795678
3,0.287824,0.286768,1.796131,1.795678
4,0.287824,0.286768,1.796131,1.795678


In [12]:
df_answers.phi1.mean()

1.7961311

In [21]:
df_opt_answers = pd.DataFrame(dataset.opt_answers.detach().numpy(),
                              columns=['alpha1_star', 'alpha2_star', 'phi1_star', 'phi2_star'])
df_hidden_states = pd.DataFrame(dataset.hidden_states.detach().numpy(),
                               columns=['m1', 'm2', 'q1', 'q2'])
df_lat_space = pd.DataFrame(lat_spaces.detach().numpy(),
                               columns=['m1_m', 'm2_m', 'q1_m', 'q2_m'])
df_question = pd.DataFrame(dataset.questions.detach().numpy(),
                           columns=['m_ref1', 'v_ref1', 'm_ref1', 'v_ref2'])

In [22]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_answers.alpha1,
                         y=df_opt_answers.alpha1_star,
                         mode='markers',
                         opacity=.5,
                         name='training loss'))
fig.show()

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_answers.alpha2,
                         y=df_opt_answers.alpha2_star,
                         mode='markers',
                         opacity=.5))
fig.show()

In [27]:
import numpy as np
.75 * np.pi

2.356194490192345

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_answers.phi2,
                         y=df_opt_answers.phi2_star,
                         mode='markers',
                         opacity=.5,
                         name='training loss'))
fig.show()

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_answers.phi1,
                         y=df_opt_answers.phi1_star,
                         mode='markers',
                         opacity=.5,
                         name='training loss'))
fig.show()

In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_lat_space.q1_m,
                     y=df_hidden_states.m1,
                     mode='markers',
                     opacity=.5,
                     name='training loss'))
fig.show()

In [28]:
selection_biases

Parameter containing:
tensor([[-0.2908, -0.2897, -0.2897, -0.2903],
        [-0.2899, -0.2898, -0.2897, -0.2898],
        [-0.2894, -0.2894, -0.2895, -0.2894],
        [-0.2894, -0.2894, -0.2894, -0.2894]], requires_grad=True)

In [85]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_lat_space.m2_m,
                         y=df_hidden_states.m2,
                         mode='markers',
                         opacity=.5,
                         name='training loss'))
fig.show()

In [16]:
df_opt_answers.values

array([[0.24239933, 0.21672727, 1.7141474 , 1.7111446 ],
       [0.29000688, 0.08630928, 1.7573335 , 1.7670382 ],
       [0.18636033, 0.09044579, 1.6711215 , 1.6756781 ],
       ...,
       [0.14206758, 0.09467524, 1.675318  , 1.6856021 ],
       [0.44811478, 0.20434587, 1.6796926 , 1.6631333 ],
       [0.2556672 , 0.14730154, 1.6516975 , 1.6467656 ]], dtype=float32)